# Random Perturbations

Streamlined notebook for evaluating random perturbations on saved models and datasets.

## Imports

In [1]:
# Standard library
import os
import sys
import time

# Third-party
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Local package imports
from minima_volume.perturb_funcs import ( analyze_wiggle_metrics_fixed_dataset )

from minima_volume.dataset_funcs import (
    load_dataset,
    load_model,
    load_models_and_data,
    prepare_datasets,
    tensor_to_list,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Input Parameters

In [2]:
# Perturbation Configuration
perturbation_seed = 1
num_directions = 500
N = 100
x = np.linspace(0, 1, N)
coefficients = x**2

base_output_dir = ""

## Model + Dataset Specific Code

This is for model and dataset specific code.

In [3]:
# User specifies the model module name
from minima_volume.models import modulo_arithmetic_model_data as model_module
modulus = 97

# MNIST specific initialization parameters
hidden_dims = [250]

# Grab model
model_template = model_module.get_model(N = modulus, hidden_dims=hidden_dims, device=device, seed=0)

# Grab loss and metrics
loss_fn = model_module.get_loss_fn()
other_metrics = model_module.get_additional_metrics()

## Loading Model and Datasets

In [4]:
# ====================================
# Load Trained Models and Dataset
# ====================================
target_dir = "models_and_data"  # relative path
loaded_models, loaded_model_data, loaded_dataset = load_models_and_data(
    model_template=model_template,
    target_dir=target_dir,
    device=device,
)

# Dataset Info
dataset_type = loaded_dataset['dataset_type']
print(f"Dataset type: {dataset_type}")
print(f"Dataset quantities: {loaded_dataset['dataset_quantities']}")

print("\nTensor shapes:")
for key in ["x_base_train", "y_base_train", "x_additional", "y_additional", "x_test", "y_test"]:
    shape = getattr(loaded_dataset[key], "shape", None)
    print(f"  {key}: {shape if shape is not None else 'None'}")

# Reconstruct trained_model dicts safely.
# If the loss or accuracy or additional metrics happen to be
# tensors, they get safely converted to lists.
all_models = [
    {
        "model": model,
        **{
            k: tensor_to_list(model_data[k], key_path=k)
            for k in ["train_loss", "train_accs", "test_loss", "test_accs", "additional_data", "dataset_type", "total_epochs_trained"]
        },
    }
    for model, model_data in zip(loaded_models, loaded_model_data)
]
print(f"Reconstructed {len(all_models)} trained models")

Looking for models and dataset in: models_and_data
Found 9 model files:
  - model_epoch_1000.pt
  - model_epoch_1500.pt
  - model_epoch_200.pt
  - model_epoch_2000.pt
  - model_epoch_3000.pt
  - model_epoch_400.pt
  - model_epoch_4000.pt
  - model_epoch_600.pt
  - model_epoch_800.pt
✅ Model loaded into provided instance from models_and_data\model_epoch_1000.pt
Successfully loaded: model_epoch_1000.pt
✅ Model loaded into provided instance from models_and_data\model_epoch_1500.pt
Successfully loaded: model_epoch_1500.pt
✅ Model loaded into provided instance from models_and_data\model_epoch_200.pt
Successfully loaded: model_epoch_200.pt
✅ Model loaded into provided instance from models_and_data\model_epoch_2000.pt
Successfully loaded: model_epoch_2000.pt
✅ Model loaded into provided instance from models_and_data\model_epoch_3000.pt
Successfully loaded: model_epoch_3000.pt
✅ Model loaded into provided instance from models_and_data\model_epoch_400.pt
Successfully loaded: model_epoch_400.pt


In [5]:
print (all_models[0]["total_epochs_trained"])
print (all_models[1]["total_epochs_trained"])
print (all_models[2]["total_epochs_trained"])
print (all_models[3]["total_epochs_trained"])

1000
1500
200
2000


In [6]:
print (all_models[0]["train_loss"][-1])
print (all_models[1]["train_loss"][-1])
print (all_models[2]["train_loss"][-1])
print (all_models[3]["train_loss"][-1])

0.0054785641841590405
0.004723789636045694
0.007694550324231386
0.00398277910426259


In [7]:
print (all_models[0]["test_loss"][-1])
print (all_models[1]["test_loss"][-1])
print (all_models[2]["test_loss"][-1])
print (all_models[3]["test_loss"][-1])

0.011248840599042987
0.009797460594562817
0.01104290118295086
0.008288254656110626


## Perturbations

Using the saved datasets, we perform model perturbations. 

In [8]:
# For evaluating against epochs, we don't consider additional datasets

 

x_base_train = loaded_dataset['x_base_train'].to(device)
y_base_train = loaded_dataset['y_base_train'].to(device)
#x_additional = loaded_dataset['x_additional'].to(device)
#y_additional = loaded_dataset['y_additional'].to(device)
x_test = loaded_dataset['x_test'].to(device)
y_test = loaded_dataset['y_test'].to(device)

# Loss function and metrics already grabbed from the model module
analyze_wiggle_metrics_fixed_dataset(
    model_list = all_models, 
    x_input = x_base_train,
    y_input = y_base_train, 
    x_test = x_test,
    y_test = y_test, 
    dataset_type = dataset_type, 
    variation_metric_key = "total_epochs_trained",
    metrics = {"loss": loss_fn, **other_metrics}, 
    coefficients = coefficients,
    num_directions = num_directions,
    perturbation_seed = perturbation_seed,
    device = device,  # can be set to GPU if needed
)

""" Our saved results are structured as follows:
wiggle_results: List of dictionaries containing wiggle test results
Each dictionary is of the form
{
'loss':
'coefficients':
'accs':
'perturbation_seed':
'perturbation_norm':
}
model: PyTorch model used in analysis (state_dict will be saved)
output_dir: Directory to save results (default: "imgs/swiss/random_dirs")
filename: Name of output file (default: "random_directions.npz")
**kwargs: Additional key-value pairs to be saved in the output file
Typically:
'additional_data':
'model_trained_data':
'dataset_type':
'base_dataset_size': 
'test_loss':
'test_accs':
'num_params':
"""

The number of parameters of the perturbation is

 73097
Testing on dataset size 3010 - 500 directions
Testing model total_epochs_trained 1000
Loss: 0.0112


Wiggle completed in 38.27 seconds for data model trained with 1000 samples
Saved to data_0

Loss: 0.0112
Accs: 0.3856


Wiggle completed in 38.35 seconds for data model trained with 1000 samples
Saved to data_0

Testing model total_epochs_trained 1500
Loss: 0.0098


Wiggle completed in 38.40 seconds for data model trained with 1500 samples
Saved to data_0

Loss: 0.0098
Accs: 0.6749


...

Wiggle completed in 38.11 seconds for data model trained with 600 samples
Saved to data_0

Loss: 0.0118
Accs: 0.3339


Wiggle completed in 38.42 seconds for data model trained with 600 samples
Saved to data_0

Testing model total_epochs_trained 800
Loss: 0.0116


Wiggle completed in 38.50 seconds for data model trained with 800 samples
Saved to data_0

Loss: 0.0116
Accs: 0.3498


Wiggle completed in 38.57 seconds for data model trained with 800 samples
Saved to data_0



' Our saved results are structured as follows:\nwiggle_results: List of dictionaries containing wiggle test results\nEach dictionary is of the form\n{\n\'loss\':\n\'coefficients\':\n\'accs\':\n\'perturbation_seed\':\n\'perturbation_norm\':\n}\nmodel: PyTorch model used in analysis (state_dict will be saved)\noutput_dir: Directory to save results (default: "imgs/swiss/random_dirs")\nfilename: Name of output file (default: "random_directions.npz")\n**kwargs: Additional key-value pairs to be saved in the output file\nTypically:\n\'additional_data\':\n\'model_trained_data\':\n\'dataset_type\':\n\'base_dataset_size\': \n\'test_loss\':\n\'test_accs\':\n\'num_params\':\n'